In [1]:
import sys
sys.path.insert(0,'../code')
import book_format
book_format.load_style('../code')

# Computational Statistics

## Distributions

In statistics a <span>**distribution**</span> is a set of values and
their corresponding probabilities.

For example, if you roll a six-sided die, the set of possible values is
the numbers 1 to 6, and the probability associated with each value is
1/6.

As another example, you might be interested in how many times each word
appears in common English usage. You could build a distribution that
includes each word and how many times it appears.

To represent a distribution, one could use two numpy arrays that
store values and their probabililties. These two arrays represent a 
<span>**probability mass function**</span> (PMF), which is a way to
represent a distribution mathematically.

To use numpy arrays the corresponding module has to be imported first:

In [1]:
import numpy as np

The following code builds two arrays to represent the distribution of
outcomes for a six-sided die as a PMF:

In [5]:
vals=np.array([1,2,3,4,5,6],dtype=float)
probs = np.full_like(vals,1/6,dtype=float)

print(vals)
print(probs)

[1. 2. 3. 4. 5. 6.]
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


The array `vals` contains the numbers on the sides of a six-sided die and the array `probs` contains the probabilities  associated with each value $1/6$.

Here’s another example that counts the number of times each word appears in a sequence:

In [9]:
word_list = np.array(['hi', 'the', 'bye', 'hi', 'football', 'sky'])
words_unique, words_counts = np.unique(word_list, return_counts=True)

print(words_unique)
print(words_counts)

['bye' 'football' 'hi' 'sky' 'the']
[1 1 2 1 1]


`np.unique` returns the unique values of an array and their counts.
The word counts are proportional to the probabilities to find each 
word in the word list. So after we count all the words, we can compute
probabilities by dividing through by the total number of words:

In [10]:
words_probs=words_counts / words_counts.sum()

print(words_unique)
print(words_probs)

['bye' 'football' 'hi' 'sky' 'the']
[0.16666667 0.16666667 0.33333333 0.16666667 0.16666667]


Once values and prbabilities are defined, one can get the probability associated with any value:

In [12]:
prob=words_probs[words_unique=='the']

print(prob)

[0.16666667]


And that would print the frequency of the word “the” as a fraction of
the words in the list.

To define a PMF two arrays with the values and their
probabilities are used, so the values in the PMF can be any hashable type. The
probabilities can be any numerical type, but they are usually
floating-point numbers (type `float`).

## The cookie problem

In the context of Bayes’s theorem, it is natural to use a Pmf to map
from each hypothesis to its probability. In the cookie problem, the
hypotheses are $B_1$ and $B_2$. In Python, I represent them with
strings:

In [14]:
prior = np.array([0.5,0.5])

This distribution, which contains the priors for each hypothesis, is
called (wait for it) the <span>**prior distribution**</span>.

To update the distribution based on new data (the vanilla cookie), we
multiply each prior by the corresponding likelihood. The likelihood of
drawing a vanilla cookie from Bowl 1 is 3/4. The likelihood for Bowl 2
is 1/2.

In [15]:
likelihood = np.array([30/40, 20/40])
product=prior*likelihood

`Mult` does what you would expect. It gets the probability for the given
hypothesis and multiplies by the given likelihood.

After this update, the distribution is no longer normalized, but because
these hypotheses are mutually exclusive and collectively exhaustive, we
can <span>**renormalize**</span>:

In [16]:
norm_constant = product.sum()
posterior = product / norm_constant

The result is a distribution that contains the posterior probability for
each hypothesis, which is called (wait now) the <span>**posterior
distribution**</span>.

Finally, we can get the posterior probability for Bowl 1:

In [17]:
print(f"The probability of the vanilla cookie came from the first bowl is {posterior[0]}")

The probability of the vanilla cookie came from the first bowl is 0.6


## The Bayesian framework

The Bayesian framework introduced in the original is not used here 
and all calculations are performed using numpy arrays.

## The Monty Hall problem

To solve the Monty Hall problem, the hypotheses and the prior are defined first:

In [20]:
hypos = np.array(['A','B','C'])
prior = np.array([1/3]*3)

Following the explanations from the previous chapter the likelihood after choosing the door B is defined as:

In [21]:
likelihood = np.array([0.5, 0, 1])

Finally, an update is performed and the posterior is calculated:

In [23]:
product = prior*likelihood
norm_constant = product.sum()
posterior = product / norm_constant
print(f"The probabilities of the car beeing behind doors {hypos} are {posterior}")

The probabilities of the car beeing behind doors ['A' 'B' 'C'] are [0.33333333 0.         0.66666667]


## Encapsulating the framework

The Bayesian framework introduced in the original is not used here and all calculations are performed using numpy arrays.

## The <span>M&M</span> problem

We can use the `Suite` framework to solve the <span>M&M</span> problem.
Writing the `Likelihood` function is tricky, but everything else is
straightforward.

In [22]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

First I need to encode the color mixes from before and after 1995:

```python
mix94 = dict(brown=30,
             yellow=20,
             ...

mix96 = dict(blue=24,
             green=20,
             ...
```

Then I have to encode the hypotheses:

```python
hypoA = dict(bag1=mix94, bag2=mix96)
hypoB = dict(bag1=mix96, bag2=mix94)
```

`hypoA` represents the hypothesis that Bag 1 is from 1994 and Bag 2 from 1996. `hypoB` is the other way around.

Next I map from the name of the hypothesis to the representation:

```python
hypotheses = dict(A=hypoA, B=hypoB)
```

And finally I can write `Likelihood`. In this case the hypothesis,
`hypo`, is a string, either `A` or `B`. The data is a tuple that
specifies a bag and a color.

        def Likelihood(self, data, hypo):
            bag, color = data
            mix = self.hypotheses[hypo][bag]
            like = mix[color]
            return like

Here’s the code that creates the suite and updates it:

In [23]:
suite = M_and_M('AB')

suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))

suite.Print()

A 0.7407407407407407
B 0.2592592592592592


The posterior probability of A is approximately $20/27$, which is what
we got before.

The code in this section is available from
<http://thinkbayes.com/m_and_m.py>. For more information see
Section [download].

## Discussion

This chapter presents the Suite class, which encapsulates the Bayesian
update framework.

<span>Suite</span> is an <span>**abstract type**</span>, which means
that it defines the interface a Suite is supposed to have, but does not
provide a complete implementation. The <span>Suite</span> interface
includes <span>Update</span> and <span>Likelihood</span>, but the
<span>Suite</span> class only provides an implementation of
<span>Update</span>, not <span>Likelihood</span>.

A <span>**concrete type**</span> is a class that extends an abstract
parent class and provides an implementation of the missing methods. For
example, <span>Monty</span> extends <span>Suite</span>, so it inherits
<span>Update</span> and provides <span>Likelihood</span>.

If you are familiar with design patterns, you might recognize this as an
example of the template method pattern. You can read about this pattern
at <http://en.wikipedia.org/wiki/Template_method_pattern>.

Most of the examples in the following chapters follow the same pattern;
for each problem we define a new class that extends <span>Suite</span>,
inherits <span>Update</span>, and provides <span>Likelihood</span>. In a
few cases we override <span>Update</span>, usually to improve
performance.

## Exercises

In Section [framework] I said that the solution to the cookie problem
generalizes to the case where we draw multiple cookies with replacement.

But in the more likely scenario where we eat the cookies we draw, the
likelihood of each draw depends on the previous draws.

Modify the solution in this chapter to handle selection without
replacement. Hint: add instance variables to <span>Cookie</span> to
represent the hypothetical state of the bowls, and modify
<span>Likelihood</span> accordingly. You might want to define a
<span>Bowl</span> object.